In [75]:
%config InlineBackend.figure_format = 'retina'

# Author: Jake Vanderplas <jakevdp@cs.washington.edu>
#
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import norm, qmc
from sklearn.neighbors import KernelDensity, NearestNeighbors, KNeighborsRegressor
from sklearn.utils.fixes import parse_version
import bisect, math

matplotlib.rcParams.update(matplotlib.rcParamsDefault)

def plt_style(use_latex = True):
    plt.rcParams["figure.dpi"] = 200
    plt.rcParams['font.family'] = "serif"
    plt.rcParams['font.size'] = 6
    plt.rcParams['text.latex.preamble'] = r'\usepackage{libertine} \usepackage[libertine]{newtxmath} \usepackage[T1]{fontenc} \usepackage{nicefrac}'
    plt.rcParams['text.usetex'] = use_latex

# density_param = {"density": True}

# # ----------------------------------------------------------------------
# # Plot the progression of histograms to kernels
# np.random.seed(1)
# N = 20
# X = np.concatenate(
#     (np.random.normal(0, 1, int(0.3 * N)), np.random.normal(5, 1, int(0.7 * N)))
# )[:, np.newaxis]
# X_plot = np.linspace(-5, 10, 1000)[:, np.newaxis]
# bins = np.linspace(-5, 10, 10)

# fig, ax = plt.subplots(1, 1, sharex=True, sharey=True)
# fig.subplots_adjust(hspace=0.05, wspace=0.05)

# # Gaussian KDE
# kde = KernelDensity(kernel="gaussian", bandwidth=0.75).fit(X)
# log_dens = kde.score_samples(X_plot)
# ax.fill(X_plot[:, 0], np.exp(log_dens), fc="#AAAAFF")
# ax.text(-3.5, 0.31, "Gaussian Kernel Density")

# ax.plot(X[:, 0], np.full(X.shape[0], -0.01), "+k")
# ax.set_xlim(-4, 9)
# ax.set_ylim(-0.02, 0.34)



In [219]:
# N = 1000
np.random.seed(1)

config = [
    {'N': 100, 'niterations': 100, 'offset_scale_rate': 0.4, 'offset_scale_const': 0.5},
    {'N': 100, 'niterations': 100, 'offset_scale_rate': 0.3, 'offset_scale_const': 0.5},
    {'N': 300, 'niterations': 300, 'offset_scale_rate': 0.4, 'offset_scale_const': 1}
][2]

N = config['N']
niterations = config['niterations']
offset_scale_rate = config['offset_scale_rate']
offset_scale_const = config['offset_scale_const']

xlim = (-3.2, 6.2)
ylim = (-0.02, 1.4)

weight = 0.5
n1 = (0, 1)
n2 = (5, 0.3)
nbar = ((1 - weight) * n1[0] + weight * n2[0], (1 - weight) * n1[1] + weight * n2[1])

X1 = norm(n1[0], n1[1]).ppf(np.linspace(0.0001, 0.9999, N))
X2 = norm(n2[0], n2[1]).ppf(np.linspace(0.0001, 0.9999, N))
Xbar = norm(nbar[0], nbar[1]).ppf(np.linspace(0.0001, 0.9999, N))

X_plot = np.linspace(xlim[0], xlim[1], 1000)
true_dens1 = norm(n1[0], n1[1]).pdf(X_plot)
true_dens2 = norm(n2[0], n2[1]).pdf(X_plot)
true_dens_bar = norm(nbar[0], nbar[1]).pdf(X_plot)

X1 = np.random.uniform(size=N)
for i in range(len(X1)):
    # X1 = np.sort(X1)
    for j in range(niterations):
        d = norm(nbar[0], nbar[1]).ppf((i + 0.5)/N)
        # offset_scale = offset_scale_const / pow(i + 1, offset_scale_rate)
        offset_scale = offset_scale_const * np.exp(-offset_scale_rate * i**0.4)
        X1[i] += offset_scale * (d - X1[i])

X2 = np.random.uniform(size=N)
sampler = qmc.Sobol(d=1, scramble=False)
for i in range(len(X2)):
    # X2 = np.sort(X2)
    for j in range(niterations):
        # d = (norm(n1[0], n1[1]).ppf((i + 0.5)/N) + norm(n2[0], n2[1]).ppf((i + 0.5)/N)) / 2
        dist = norm(n1[0], n1[1]) if np.random.uniform() >= weight else norm(n2[0], n2[1])
        # dist = norm(n1[0], n1[1]) if sampler.random() >= weight else norm(n2[0], n2[1])
        d = dist.ppf((i + 0.5)/N)
        # offset_scale = offset_scale_const / pow(i + 1, offset_scale_rate)
        offset_scale = offset_scale_const * np.exp(-offset_scale_rate * i**0.4)
        X2[i] += offset_scale * (d - X2[i])

In [294]:
plt_style()

fig, ax = plt.subplots()

ax.fill(X_plot, true_dens1, fc=(0.47, 0.69, 1.00), alpha=0.35, label="Input densities")
ax.fill(X_plot, true_dens2, fc=(0.47, 0.69, 1.00), alpha=0.35, label="Input densities")
ax.fill(X_plot, true_dens_bar, fc=(0.47, 0.69, 1.00), alpha=0.2, label="Target barycenter")

# datasets = zip([X1], ["blue"], ["Explicit barycenter $\overline{\mu}$ optimization"])
# datasets = zip([X2], ["magenta"], ["SGD barycenter $\overline{\mu}$ optimization"])
datasets = zip([X1, X2], [(1, 0.4, 0.4), (0.4, 0.65, 0.3)], ["Exact-gradient optimization", "Stochastic-gradient optimization"])

def KnnDensity(X, query, nneighbors=30, max_radius=0.4):
    neighbors = []
    farthest_dist = max_radius
    for x in X:
        if abs(query - x) < farthest_dist:
            bisect.insort(neighbors, x)
            if len(neighbors) > nneighbors:
                neighbors.pop(0 if abs(query - neighbors[0]) > abs(query - neighbors[-1]) else -1)
            farthest_dist = max(abs(query - neighbors[0]), abs(query - neighbors[-1]))
    return 3/4 * sum([1 - (abs(query - x) / farthest_dist)**2 for x in neighbors]) / (farthest_dist * len(X))
    # return (len(neighbors) - 1) / (2 * farthest_dist * len(X))
    # return neighbors

for X, color, name in datasets:
    density = np.exp(KernelDensity(kernel="gaussian", bandwidth=0.09).fit(X[:, np.newaxis]).score_samples(X_plot[:, np.newaxis]))
    # density = [KnnDensity(X, query, 10, 0.5) for query in X_plot]
    ax.plot(X_plot, density, color=color, lw=1.3, linestyle="-", label=name)
    # ax.plot(X, -0.005 - 0.01 * np.random.random(X.shape[0]), "+", color=color) # Plot individual points

# ax.text(6, 0.38, "N={0} points".format(N))


handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), loc="upper left", bbox_to_anchor=(-0.02,1.02), frameon=False, prop={'family': 'Linux Libertine', 'size': 7}, labelspacing = 0.2)
# ax.legend(by_label.values(), by_label.keys())

# ax.set_facecolor('xkcd:salmon')
# fig.patch.set_facecolor((0.96, 0.96, 0.96))
ax.set_xlim(xlim)
ax.set_ylim(ylim)
ax.axis('off')

fig.set_size_inches(3.377 - 0.06, 1.2)
plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, hspace = 0, wspace = 0)

plt.savefig('1D-barycenter.pdf', bbox_inches='tight', pad_inches=0.02)
plt.show()

In [185]:
query = 60
neighbors = [1,10,30,45]
bisect.insort(neighbors, 50)
# a.pop(0 if abs(query - a[0]) > abs(query - a[-1]) else -1)
neighbors.pop(0 if abs(query - neighbors[0]) > abs(query - neighbors[-1]) else -1)
neighbors

i = bisect.bisect_left(neighbors, query)
dist_hi = abs(neighbors[i] - query) if i < len(neighbors) else math.inf
dist_lo = abs(query - neighbors[i-1]) if i > 0 else math.inf
nearest_dist = min(dist_lo, dist_hi)
print(neighbors, nearest_dist)

[10, 30, 45, 50] 10


In [192]:
KnnDensity(X1, 2.1)
# print(X_plot)

[2.0089802276426925,
 2.030388610117852,
 2.0512992565895583,
 2.0717554884331433,
 2.091796063443698,
 2.1114558790282363,
 2.1307664753700046,
 2.1497565357503463,
 2.1684521557879677,
 2.1868774379551073]

In [198]:
sum([1,2,3])

6